### MicroPath Reconstruction From AIS Broadbast Points

Micropathing is the construction of a target's path from a limited set of a consecutive sequence of target points. Typically, the sequence is time-based, and the collection is limited to 2 or 3 target points.  The following is an illustration of 2 micropaths derived from 3 target points:

![](media/Micropath0.png)

Micropathing is different than path reconstruction, in such that the latter produced one polyline for the path of a target. Path reconstruction losses insightful in-path behavior, as a large number of attributes cannot be associated with the path parts. Some can argue that the points along the path can be enriched with these attributes. However, with the current implementations of Point objects, we are limited to only the extra `M` and `Z` to the necessary `X` and `Y`. You can also join the `PathID` and `M` to a lookup table and gain back that insight, but that joining is typically expensive and is difficult to derive from it the "expression" of the path using traditional mapping. A micropath overcomes today's limitations with today's traditional means to express the path insight better.

So, a micropath is a line composed of typically 2 points only and is associated with a set of attributes that describe that line.  These attributes are typical enrichment metrics derived from its two ends. An attribute can be, for example, the traveled distance, time, or speed.

In this notebook, we will construct "clean" micropaths using SparkSQL.  What do I mean by clean? As we all know, emitted target points are notoriously affected by noise, so using SparkSQL, we will eliminate that noise during the micropath construction.

In [8]:
import os
import arcpy
from spark_esri import spark_start, spark_stop

In [ ]:
config = {"spark.driver.memory":"2G"}
spark = spark_start(config=config)

In [94]:
sp_ref = arcpy.SpatialReference(3857)
fields = ["MMSI","SHAPE@X","SHAPE@Y","BaseDateTime"]

In [95]:
with arcpy.da.SearchCursor("Broadcast", fields, spatial_reference=sp_ref) as rows:
    spark\
        .createDataFrame(rows, "mmsi string,x double,y double,t timestamp")\
        .selectExpr("mmsi","x","y","hour(t) h","unix_timestamp(t) t")\
        .createOrReplaceTempView("v0")

In [96]:
spark\
    .sql("""
select mmsi,h,
x x1,
y y1,
t t1,
lead(x,1,0.0) over (partition by mmsi order by t) x2,
lead(y,1,0.0) over (partition by mmsi order by t) y2,
lead(t,1,0) over (partition by mmsi order by t) t2
from v0
""")\
    .createOrReplaceTempView("v1")

In [97]:
spark.sql("select *,(x2-x1) dx,(y2-y1) dy,(t2-t1) dt from v1 where t1 < t2").createOrReplaceTempView("v2")

In [98]:
spark.sql("select mmsi,h,x1,y1,x2,y2,sqrt(dx*dx+dy*dy) dd,dt from v2").createOrReplaceTempView("v3")

In [99]:
spark.sql("select *,dd/dt mps from v3").createOrReplaceTempView("v4")

In [106]:
spark.sql("""
select
percentile_approx(mps,0.99) mps,
percentile_approx(dt,0.99) dt,
percentile_approx(dd,0.99) dd
from v4
""").show()

+-----------------+---+-----------------+
|              mps| dt|               dd|
+-----------------+---+-----------------+
|12.32537246432986|180|778.6552542836735|
+-----------------+---+-----------------+



In [84]:
rows = spark.sql("""
select mmsi,h,dd,dt,mps,concat('LINESTRING(',x1,' ',y1,',',x2,' ',y2,')') wkt
from v4
where dd between 1 and 1500
and mps < 25
and dt < 130
""")\
    .collect()

In [85]:
ws = "memory"
nm = "MicroPaths"

fc = os.path.join(ws,nm)

arcpy.management.Delete(fc)

sp_ref = arcpy.SpatialReference(3857)
arcpy.management.CreateFeatureclass(ws,nm,"POLYLINE",spatial_reference=sp_ref)
arcpy.management.AddField(fc, "MMSI", "TEXT")
arcpy.management.AddField(fc, "HH", "LONG")
arcpy.management.AddField(fc, "DD", "DOUBLE")
arcpy.management.AddField(fc, "DT", "DOUBLE")
arcpy.management.AddField(fc, "MPS", "DOUBLE")

with arcpy.da.InsertCursor(fc, ["MMSI","HH","DD","DT","MPS","SHAPE@WKT"]) as cursor:
    for row in rows:
        cursor.insertRow(row)

In [ ]:
spark_stop()